In [16]:
import pandas as pd
import numpy as np
from covid.simulation import create_population
from covid.utils import create_named_tuple

# read filepaths 
paths = create_named_tuple('filepaths.txt')
population = create_population(paths.muncipalities_names, paths.muncipalities_pop)

In [45]:
def generate_ssb_od_matrix(num_time_steps, population, fpath_muncipalities_commute):
    """ generate an OD-matrix used for illustrative purposes only

    Parameters
        num_regions: int indicating number of regions e.g 356
        num_time_steps: int indicating number of time periods e.g 28
    Returns
        An OD-matrix with dimensions (num_time_steps, num_regions, num_regions) with 0.8 on its                 diagonal and 0.1 on cells next to diagonal 
    """

    df = pd.read_csv(fpath_muncipalities_commute, usecols=[1,2,3])
    df['from'] = df['from'].str.lstrip("municip municip0").astype(int)
    df['to'] = df['to'].str.lstrip("municip municip0").astype(int)
    decision_period = 28
    region_id = population.region_id.to_numpy()
    od = np.zeros((decision_period, len(region_id), len(region_id)))

    morning_travel = np.zeros((len(region_id), len(region_id)))
    for id in region_id:
        i = np.where(region_id == id)
        filtered = df.where(df["from"] == id).dropna()
        to, n = filtered['to'].to_numpy(int), filtered['n'].to_numpy()
        for k in range(len(to)):
            j = np.where(region_id == to[k])
            morning_travel[i,j] = n[k]

    for i in range(morning_travel.shape[0]):
        sum_travel = morning_travel[i].sum()
        pop_i = population.iloc[i].population
        morning_travel[i,i] = pop_i - sum_travel
    
    afternoon_travel = morning_travel.T
    
    morning_stay = np.zeros((len(region_id), len(region_id)))
    np.fill_diagonal(morning_stay, 
    afternoon_stay = np.zeros((len(region_id), len(region_id)))

    for i in range(num_time_steps):
        if (i)%4 == 0: #0000-0600
            od[i] = morning_travel
        elif (i-1)%4 == 0: #0600-1200
            od[i] = morning_travel
        elif (i-2)%4 == 0: #1200-1800
            od[i] = afternoon_travel
        elif (i-3)%4 == 0: #1800-0000
            od[i] = afternoon_travel
    return od, morning_travel, afternoon_travel

In [46]:
od, m, a = generate_ssb_od_matrix(28, population, paths.municipalities_commute)


In [60]:

print(m[:,0].sum())
print(m)

582395.0
[[5.19882e+05 7.10000e+01 1.42300e+03 ... 2.00000e+00 5.00000e+00
  5.40000e+01]
 [1.30000e+01 1.36920e+04 5.80000e+01 ... 0.00000e+00 0.00000e+00
  0.00000e+00]
 [7.50000e+02 4.10000e+02 1.12863e+05 ... 0.00000e+00 2.00000e+00
  3.00000e+00]
 ...
 [0.00000e+00 0.00000e+00 0.00000e+00 ... 8.56000e+02 0.00000e+00
  8.00000e+00]
 [4.00000e+00 0.00000e+00 0.00000e+00 ... 3.00000e+00 2.16300e+03
  3.00000e+00]
 [1.80000e+01 0.00000e+00 1.00000e+00 ... 1.20000e+01 8.00000e+00
  9.81900e+03]]
